In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import BertModel, DistilBertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
import random
import os
import seaborn as sns


### Part A

In [2]:
# set device to cuda if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
data = []
max_length = 0

i=0
with open("abusaeed_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 0))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 
                i+=1
            except StopIteration:
                break


i=0
with open("amir_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 1))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break


i=0
with open("anvari_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 2))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break
        

i=0
with open("asad_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 3))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break

i=0
with open("asadi_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 4))

                max_length = len(B11.split(' '))if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break


i=0
with open("attar_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 5))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break


i=0
with open("babaafzal_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 6))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break


i=0
with open("bahaee_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 7))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break


i=0
with open("bahar_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 8))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break


i=0
with open("bidel_norm.txt", encoding="utf8") as file:
    next(file)
    next(file)
    for line in file:
        if i<1000:
            try:
                B11 = line.strip()
                if B11=='':
                    B11 = next(file).strip()
                data.append((B11, 8))

                max_length = (len(B11.split(' '))) if (len(B11.split(' '))) > max_length else max_length 

                i+=1
            except StopIteration:
                break
MAX_LENGTH = max_length+2

In [4]:
train_data, test_data = train_test_split(data, test_size=0.3)

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained(
    'HooshvareLab/distilbert-fa-zwnj-base'
)

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text, label = self.data[index]
        
        text = tokenizer(
            text,
            padding='max_length',
            max_length=512,
            truncation=True,
            return_tensors='pt',
        )
        
        return text, label

In [6]:
train_data = MyDataset(train_data)
test_data = MyDataset(test_data)

In [7]:
class BertClassifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.bert= BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768, 10),
            nn.ReLU(),
        )
        
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        output = self.linear_relu_stack(pooled_output)
        return output

In [8]:
model = BertClassifier().to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
for param in model.bert.parameters():
    param.requires_grad = False

In [10]:
learning_rate = 5e-3
batch_size = 64
epochs = 1

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [12]:
from torch.optim import Adam
from tqdm import tqdm


def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_correct = 0
    for batch_num, (X, y) in enumerate(dataloader):
        losses = []
        # forward prop
        y = y.to(device)
        mask = X['attention_mask'].to(device)
        input_id = X['input_ids'].squeeze(1).to(device)
        output = model(input_id, mask)
        loss = loss_fn(output, y)
        losses.append(loss.item())
        num_correct += (output.argmax(dim=1) == y).sum().item()
        # back prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_num % 25 == 0:
            print(f'batch: {batch_num}, loss: {np.mean(losses)}')
            losses = []
    
    accuracy = num_correct / size
    print(f'train_loss: {loss.item()}')
    print(f'Train Accuracy: {np.round(accuracy*100,2)}%')
    

def train(model, train_dataloader, test_dataloader, learning_rate, epochs, optimizer):
    model.train()
    size = len(train_dataloader.dataset)
    for epoch_num in range(epochs):
        num_correct = 0
        total_acc_train = 0
        total_loss_train = 0
        for train_input, train_label in tqdm(train_dataloader):
            losses = []
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)
            print(mask.device)
            print(input_id.device)
            model = model.to(device)
            output = model(input_id, mask).to(device)

            loss = loss_fn(output, y.to(device))
            losses.append(loss.item())
            num_correct += (output.argmax(dim=1) == y).sum().item()
        
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for test_input, test_label in test_dataloader:

                test_label = test_label.to(device)
                mask = test_input['attention_mask'].to(device)
                input_id = test_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask).to(device)

                batch_loss = criterion(output, test_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == test_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(test_data): .3f} \
            | Val Accuracy: {total_acc_val / len(test_data): .3f}')



In [13]:
EPOCHS = 1
model = BertClassifier()
LR = 1e-6
train(model, train_dataloader, test_dataloader, LR, 1, optimizer)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                          | 0/109 [00:

cuda:0
cuda:0


  0%|                                                                                          | 0/109 [00:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 2.00 GiB total capacity; 1.62 GiB already allocated; 0 bytes free; 1.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Due to processing limitations, we were unable to fully train the network and then calculate confusion matrix and ...

In [ ]:
## F1-Score, ...:
print(classification_report(actual_labels, estimated_labels))


### Confusion matrix:
confmat = confusion_matrix(actual_labels, estimated_labels)
sns.heatmap(
    confmat,
    xticklabels=label_map.values(),
    yticklabels=label_map.values(),
    annot=True,
    fmt='d'
);

#### Part B

In [ ]:
model = BertClassifier().to(device)

In [ ]:
learning_rate = 5e-3
batch_size = 64
epochs = 1

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
EPOCHS = 1
model = BertClassifier()
LR = 1e-6
train(model, train_dataloader, test_dataloader, LR, 1, optimizer)

Again, due to processing limitations, we were unable to fully train the network and then calculate confusion matrix and ...

In [ ]:
## F1-Score, ...:
print(classification_report(actual_labels, estimated_labels))


### Confusion matrix:
confmat = confusion_matrix(actual_labels, estimated_labels)
sns.heatmap(
    confmat,
    xticklabels=label_map.values(),
    yticklabels=label_map.values(),
    annot=True,
    fmt='d'
);